In [53]:
import funciones_etl
etl = funciones_etl
import pandas as pd

**------------------------------------------User Review------------------------------------------**

In [54]:
df_review = etl.Cargar_a_Dataframe('datos_originales/user_reviews.json.gz','rb')
etl.CalcularNulos(df_review)

el porcentaje de datos nulos en la columna  user_id  es del  0.0 %
el porcentaje de datos nulos en la columna  user_url  es del  0.0 %
el porcentaje de datos nulos en la columna  reviews  es del  0.0 %
-------------------------------------------------------------------------------------------


In [55]:
df_review.head()

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."


In [56]:
df_review = etl.DesanidarColumnas(df_review,'reviews','user_id')
etl.CalcularNulos(df_review)
df_review.head()

el porcentaje de datos nulos en la columna  user_id  es del  0.0 %
el porcentaje de datos nulos en la columna  funny  es del  0.0 %
el porcentaje de datos nulos en la columna  posted  es del  0.0 %
el porcentaje de datos nulos en la columna  last_edited  es del  0.0 %
el porcentaje de datos nulos en la columna  item_id  es del  0.0 %
el porcentaje de datos nulos en la columna  helpful  es del  0.0 %
el porcentaje de datos nulos en la columna  recommend  es del  0.0 %
el porcentaje de datos nulos en la columna  review  es del  0.0 %
-------------------------------------------------------------------------------------------


,user_id,funny,posted,last_edited,item_id,helpful,recommend,review
0,76561197970982479,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,76561197970982479,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.
2,76561197970982479,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
4,js41637,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...


se eliminan columnas pocos relevantes: funny, helpful, recommend

In [57]:
df_review = df_review.drop(['funny', 'helpful'], axis=1)
df_review.head()

,user_id,posted,last_edited,item_id,recommend,review
0,76561197970982479,"Posted November 5, 2011.",,1250,True,Simple yet with great replayability. In my opi...
1,76561197970982479,"Posted July 15, 2011.",,22200,True,It's unique and worth a playthrough.
2,76561197970982479,"Posted April 21, 2011.",,43110,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,"Posted June 24, 2014.",,251610,True,I know what you think when you see this title ...
4,js41637,"Posted September 8, 2013.",,227300,True,For a simple (it's actually not all that simpl...


In [58]:
df_review['last_edited'].value_counts()

last_edited
                                   53165
Last edited November 25, 2013.        99
Last edited October 17, 2015.         19
Last edited June 6, 2015.             18
Last edited January 3.                17
                                   ...  
Last edited May 21, 2015.              1
Last edited September 16, 2015.        1
Last edited March 30, 2014.            1
Last edited March 24.                  1
Last edited August 15, 2014.           1
Name: count, Length: 1015, dtype: int64

In [59]:
df_review.to_parquet('datos/reviews.parquet',index=False)

**--------------------------------------------------------------------------------------------------**

In [60]:
review = pd.read_parquet('datos/reviews.parquet')

In [61]:
print(review.shape)
review.head()

(59333, 6)


,user_id,posted,last_edited,item_id,recommend,review
0,76561197970982479,"Posted November 5, 2011.",,1250,True,Simple yet with great replayability. In my opi...
1,76561197970982479,"Posted July 15, 2011.",,22200,True,It's unique and worth a playthrough.
2,76561197970982479,"Posted April 21, 2011.",,43110,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,"Posted June 24, 2014.",,251610,True,I know what you think when you see this title ...
4,js41637,"Posted September 8, 2013.",,227300,True,For a simple (it's actually not all that simpl...


eliminamos cualquier fila de la columna 'posted' que no contenga una reseña 

In [62]:
review = review.dropna(subset=['posted'])

**utilizamos como referencia la ultima publicacion, ya que suele ocurrir que la primera reseña fue solo un analisis de los primeros minutos de juego, y que cuando tienen un chequeo mas detallado** (*luego de haber pasado mas tiempo jugando*) **modifican su opinion**

In [63]:
review.loc[review['last_edited'] != '', 'posted'] = review['last_edited']

In [64]:
review = etl.ExtraerAño(review,'posted','year_posted')

/home/MJB/Desktop/PI-steam/etl/funciones_etl.py:52: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df = df[df[columna].str.contains(year_r)]
/home/MJB/Desktop/PI-steam/etl/funciones_etl.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[nueva_columna] = df[columna].str.extract(year_r)


In [65]:
review.head()

,user_id,last_edited,item_id,recommend,review,year_posted
0,76561197970982479,,1250,True,Simple yet with great replayability. In my opi...,2011
1,76561197970982479,,22200,True,It's unique and worth a playthrough.,2011
2,76561197970982479,,43110,True,Great atmosphere. The gunplay can be a bit chu...,2011
3,js41637,,251610,True,I know what you think when you see this title ...,2014
4,js41637,,227300,True,For a simple (it's actually not all that simpl...,2013
